In [ ]:
import pandas as pd

pd_siniestros = pd.read_excel('../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx')
excel_path = '../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx'

In [3]:
###
# 1. Ver todas las hojas
###
print('Hojas disponibles en el archivo:')
hojas = pd.ExcelFile(excel_path).sheet_names
for i, hoja in enumerate(hojas):
    print(f"   {i+1}. {hoja}")

Hojas disponibles en el archivo:
   1. Siniestros
   2. Vehiculos
   3. Actor_vial
   4. Limpieza de datos
   5. Diccionario


In [5]:
###
# 2. Filtrar hojas que no quieres (apuntes, tareas)
###
hojas_interes = []
for hoja in hojas:
    if 'siniestros' in hoja.lower() or 'limpieza' in hoja.lower() or 'diccionario' in hoja.lower():
        continue
    else:
        hojas_interes.append(hoja)

print(f'\nHojas a analizar: {hojas_interes}')


Hojas a analizar: ['Vehiculos', 'Actor_vial']


In [ ]:
###
# 3. Ver todas las columnas de cada hoja de interes (función)
###
def analizar_hoja_detallada(hoja_nombre):
    '''Función para análisis detallado de cada hoja'''
    separador = '=' * 60
    print(f'\n{separador}')
    print(f'Analisis detallado - hoja: {hoja_nombre}')
    print(f'{separador}')
    
    # Cargar la hoja
    df_sinistros_clean = pd.read_excel(excel_path, sheet_name=hoja_nombre)
    
    # Información básica
    print(f'Dimensiones: {df_sinistros_clean.shape[0]} filas × {df_sinistros_clean.shape[1]} columnas')
    
    # Columnas
    print(f'\nLista de Columnas ({len(df_sinistros_clean.columns)} total):')
    for i, columna in enumerate(df_sinistros_clean.columns, 1):
        print(f'   {i:2d}. {columna}')
    
    # Tipos de datos
    print(f'\nTipo de dátos:')
    print(df_sinistros_clean.dtypes)
    
    # Valores nulos
    print(f'\nValores nulos (primeras 10 columnas):')
    nulos = df_sinistros_clean.isnull().sum()
    print(nulos)
    
    return df_sinistros_clean



In [ ]:
###
# 3. Ver todas las columnas de cada hoja de interes (ejecución)
###
dataframes = {}
for hoja in hojas_interes:
    dataframes[hoja] = analizar_hoja_detallada(hoja)


Analisis detallado - hoja: Vehiculos
Dimensiones: 523168 filas × 32 columnas

Lista de Columnas (32 total):
    1. Codigo_Accidente
    2. Formulario
    3. Fecha_Acc
    4. AA_Acc
    5. Codigo_Vehiculo
    6. Clase
    7. Servicio
    8. Modalidad
    9. Vehiculo_Viajaba_Clasificado
   10. Tipo_SITP
   11. Con_Bicicleta
   12. Con_Carga
   13. Con_Embriaguez
   14. Con_Huecos
   15. Con_Menores
   16. Con_Moto
   17. Con_Peaton
   18. Con_Persona_Mayor
   19. Con_Rutas
   20. Con_Tpi
   21. Con_Tpp
   22. Con_Velocidad
   23. Con_Sitp
   24. Con_Troncal
   25. Con_Alimentador
   26. Con_Zonal
   27. Con_Provisional
   28. Con_Articulado
   29. Con_Biarticulado
   30. Con_Padron_Dual
   31. Con_Servicio_Especial
   32. Con_Taxi

Tipo de dátos:
Codigo_Accidente                         int64
Formulario                              object
Fecha_Acc                       datetime64[ns]
AA_Acc                                   int64
Codigo_Vehiculo                        float64
Clase    

In [ ]:
###
# 4. Tabla de analisis para valores unicos vs nulos (función)
###
def generar_tabla_analisis(hoja_nombre):
    """Genera solo la tabla de análisis sin resumen"""
    df = dataframes[hoja_nombre]
    
    separador = '=' * 80
    print(f'\n{separador}')
    print(f'Tabla de Análisis - Hoja: {hoja_nombre}')
    print(f'{separador}')
    
    resultados = []
    
    for columna in df.columns:
        unique_vals = df[columna].nunique()
        total_vals = df.shape[0]
        nulos = df[columna].isnull().sum()
        
        pct_nulos = round((nulos / total_vals * 100), 2) if total_vals > 0 else 0
        pct_unicos = round((unique_vals / total_vals * 100), 2) if total_vals > 0 else 0
        tipo_dato = df[columna].dtype
        
        # Evaluar utilidad
        if pct_nulos > 80:
            utilidad = 'ELIMINAR'
        elif unique_vals == 1:
            utilidad = 'ELIMINAR'
        elif pct_unicos > 90:
            utilidad = 'REVISAR'
        else:
            utilidad = 'MANTENER'
        
        resultados.append({
            'Columna': columna,
            'Tipo': tipo_dato,
            'Unicos': unique_vals,
            '% Unicos': pct_unicos,
            'Nulos': nulos,
            '% Nulos': pct_nulos,
            'Utilidad': utilidad
        })
    
    df_resultados = pd.DataFrame(resultados)
    print(df_resultados.to_string(index=False))
    
    return df_resultados



In [ ]:
###
# 4. Tabla de analisis para valores unicos vs nulos (ejecución)
###
for hoja in hojas_interes:
    tabla_resultados = generar_tabla_analisis(hoja)


Tabla de Análisis - Hoja: Vehiculos
                     Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidente          int64  278466     53.23      0     0.00 MANTENER
                  Formulario         object  278466     53.23      0     0.00 MANTENER
                   Fecha_Acc datetime64[ns]    3653      0.70      0     0.00 MANTENER
                      AA_Acc          int64      10      0.00      0     0.00 MANTENER
             Codigo_Vehiculo        float64      30      0.01      3     0.00 MANTENER
                       Clase         object      22      0.00   4023     0.77 MANTENER
                    Servicio         object       5      0.00      0     0.00 MANTENER
                   Modalidad         object       8      0.00 421229    80.52 ELIMINAR
Vehiculo_Viajaba_Clasificado         object       8      0.00      0     0.00 MANTENER
                   Tipo_SITP         object       8      0.00 475607    90.91 ELIMINAR
      

In [ ]:
###
# Filtrado de eliminar y revisar (ejecución)
for hoja in hojas_interes:
    tabla_resultados = generar_tabla_analisis(hoja)
    
    # Mostrar solo las problemáticas
    problematicas = tabla_resultados[tabla_resultados['Utilidad'] != 'MANTENER']
    if not problematicas.empty:
        print(f'\n--- Columnas problemáticas en {hoja} ---')
        print(problematicas.to_string(index=False))


Tabla de Análisis - Hoja: Vehiculos
                     Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidente          int64  278466     53.23      0     0.00 MANTENER
                  Formulario         object  278466     53.23      0     0.00 MANTENER
                   Fecha_Acc datetime64[ns]    3653      0.70      0     0.00 MANTENER
                      AA_Acc          int64      10      0.00      0     0.00 MANTENER
             Codigo_Vehiculo        float64      30      0.01      3     0.00 MANTENER
                       Clase         object      22      0.00   4023     0.77 MANTENER
                    Servicio         object       5      0.00      0     0.00 MANTENER
                   Modalidad         object       8      0.00 421229    80.52 ELIMINAR
Vehiculo_Viajaba_Clasificado         object       8      0.00      0     0.00 MANTENER
                   Tipo_SITP         object       8      0.00 475607    90.91 ELIMINAR
      